# Illustrate the $\lambda$-function approach to curve fitting

Allows one to "easily" hold parameters fixed in a curve fit.  The goal is to do this without having to write a separate fit function for each combination of parameters (some fixed, some free).  The trick is useful for complex fits where initial guesses might be hard to figure out.  You make "reasonable" guesses for the parameters, fix some and let the fit program optimize the rest.  Then (maybe) the full fit will converge.

### First:  import needed packages

In [1]:
from scipy.optimize import curve_fit
import numpy as np

### Get the data

In [2]:
xdata = np.array([1, 2, 3, 4, 5, 6])                      # <... load your data here ...>
ydata = xdata**2                                          # <... load your data here ...> 

def func(x, a, b, c):
	return a*x + b*x**2 + c

# Curve fit with all parameters free
popt, pcov = curve_fit(func, xdata, ydata, p0=[1, 2, 3])
print(popt)

[-2.1795856e-12  1.0000000e+00 -6.5480954e-12]


/opt/conda/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Now let's do a fit where the second parameter is held fixed.  The trick is to use a "lambda function".  This is a simple function defined inline.  See this [link](https://www.educative.io/blog/python-lambda-functions-tutorial) for a quick background.

Here, we use a lambda function to fix a parameter in a curve fit.  The idea is to define a _new_ function "on the fly" (=lambda function) that has one less variable than the original function.  Here, we 

### fix b and fit only a and c

In [3]:
popt, pcov = curve_fit(lambda x, a, c: func(x, a, 10.0, c), xdata, ydata, p0=[1, 3])
print(popt)

[-63.  84.]


This is equivalent to defining a new function (something we would rather not have to do!):

In [4]:
def func1(x, a, c):
	return func(x, a, 10.0, c)
	
popt, pcov = curve_fit(func1, xdata, ydata, p0=[1, 3])
print(popt)

[-63.  84.]


It is also equivalent to

In [5]:
func1 = lambda x, a, c: func(x, a, 10.0, c)

popt, pcov = curve_fit(func1, xdata, ydata, p0=[1, 3])
print(popt)

[-63.  84.]


### A more advanced way of specifying parameters:

In [6]:
def fixed_param2_generator(b):
	return lambda x, a, c: func(x, a, b, c)

popt, pcov = curve_fit(fixed_param2_generator(10.0), xdata, ydata, p0=[1, 3])
print(popt)

[-63.  84.]


### An even more advanced approach:

The following approximates the behaviour of a fit package known as [lmfit](https://lmfit.github.io/lmfit-py/) that is currently not supported by Syzygy.

And even if it were, its learning curve is a bit steep....  An advantage is that you have to specify the possibilities only once.

Note that if you really need this kind of flexibility, commercial programs such as [Igor Pro](https://www.wavemetrics.com/) are _much_ easier to use!

In [7]:
def func_with_fixed_params(a=None, b=None, c=None):
	if a is not None and b is not None and c is not None:
		raise ValueError('Cannot fix all parameters')
	elif a is not None and b is not None:
		return lambda x, c: func(x, a, b, c)
	elif a is not None and c is not None:
		return lambda x, b: func(x, a, b, c)
	elif b is not None and c is not None:
		return lambda x, a: func(x, a, b, c)
	elif a is not None:
		return lambda x, b, c: func(x, a, b, c)
	elif b is not None:
		return lambda x, a, c: func(x, a, b, c)
	elif c is not None:
		return lambda x, a, b: func(x, a, b, c)
	else:
		return func
    
# Some examples using the function
popt, pcov = curve_fit(func_with_fixed_params(b=10.0), xdata, ydata, p0=[1, 3])
print(popt)
popt, pcov = curve_fit(func_with_fixed_params(a=1.0, b=10.0), xdata, ydata, p0=[3])
print(popt)
popt, pcov = curve_fit(func_with_fixed_params(c=0.0), xdata, ydata, p0=[1, 2])
print(popt)
popt, pcov = curve_fit(func_with_fixed_params(), xdata, ydata, p0=[1, 2, 3])
print(popt)

[-63.  84.]
[-140.]
[-2.18425278e-12  1.00000000e+00]
[-2.1795856e-12  1.0000000e+00 -6.5480954e-12]
